## Метод .groupby() как способ построения сводных таблиц

In [2]:
# импортируюем библиотеку
import pandas as pd


# читаем csv-файл
melb_df = pd.read_csv('data/melb_data_fe.csv', sep=',')
display()

# выводим все предыдущие изменения таблицы
# меняю тип признака
melb_df['Date'] = pd.to_datetime(melb_df['Date'])

# помещаю ряд в переменную для подготовки признака
quarter_column = melb_df['Date'].dt.quarter

# создаю список исключений
exclude_list = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car']
# создаю переменную для фильтрации
max_unique_count = 150
# расматриваем каждый признак
for col in melb_df.columns:
    # условием выбираем нужные признаки
    if melb_df[col].nunique() < max_unique_count and col not in exclude_list:
        melb_df[col] = melb_df[col].astype('category')

In [3]:
# строим таблицу которая показывает зависимость медианной цены и площади 
# здания от количества комнат
melb_df.groupby(by='Rooms')[['Price', 'BuildingArea']].median()


,Price,BuildingArea
Rooms,,
1,385000.0,107.0
2,690000.0,126.0
3,950000.0,126.0
4,1285000.0,142.0
5,1660000.0,176.0
6,1800000.0,126.0
7,1496000.0,216.5
8,1515000.0,126.0
10,900000.0,126.0


In [4]:
# расмотрим не только количество комнат, но и тип здания
melb_df.groupby(by=['Rooms', 'Type'])['Price'].median()

/tmp/ipykernel_12621/2338397871.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  melb_df.groupby(by=['Rooms', 'Type'])['Price'].median()


Rooms  Type     
1      house         857500.0
       townhouse     580000.0
       unit          377500.0
2      house         950000.0
       townhouse     675000.0
       unit          585000.0
3      house         990000.0
       townhouse     900000.0
       unit          780000.0
4      house        1300000.0
       townhouse    1157500.0
       unit          830000.0
5      house        1670000.0
       townhouse    1000000.0
       unit               NaN
6      house        1820000.0
       townhouse          NaN
       unit          520000.0
7      house        1496000.0
       townhouse          NaN
       unit               NaN
8      house        1150000.0
       townhouse          NaN
       unit         2250000.0
10     house         900000.0
       townhouse          NaN
       unit               NaN
Name: Price, dtype: float64

### .unstack()

In [5]:
# переопределяем один из вложенных индексов в виде столбца таблицы
melb_df.groupby(['Rooms', 'Type'])['Price'].median().unstack()

/tmp/ipykernel_12621/3472271481.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  melb_df.groupby(['Rooms', 'Type'])['Price'].median().unstack()


Type,house,townhouse,unit
Rooms,,,
1,857500.0,580000.0,377500.0
2,950000.0,675000.0,585000.0
3,990000.0,900000.0,780000.0
4,1300000.0,1157500.0,830000.0
5,1670000.0,1000000.0,NaN
6,1820000.0,NaN,520000.0
7,1496000.0,NaN,NaN
8,1150000.0,NaN,2250000.0
10,900000.0,NaN,NaN


## Метод pivot_table() для построения сводных таблиц



        values — имя столбца, по которому необходимо получить сводные данные, применяя агрегирующую функцию;
        index — имя столбца, значения которого станут строками сводной таблицы;
        columns — имя столбца, значения которого станут столбцами сводной таблицы;
        aggfunc — имя или список имён агрегирующих функций (по умолчанию — подсчёт среднего, 'mean');
        fill_value — значение, которым необходимо заполнить пропуски (по умолчанию пропуски не заполняются).



In [7]:
# строим сводную таблицу и заменим пропущенные значения на ноль
melb_df.pivot_table(
    values='Price',
    index='Rooms',
    columns='Type',
    fill_value=0
).round()

Type,house,townhouse,unit
Rooms,,,
1,866866.0,592705.0,389929.0
2,1017238.0,710158.0,610491.0
3,1109233.0,984709.0,850596.0
4,1462283.0,1217092.0,1037476.0
5,1877327.0,1035000.0,0.0
6,1869508.0,0.0,520000.0
7,1920700.0,0.0,0.0
8,1510286.0,0.0,2250000.0
10,900000.0,0.0,0.0


In [8]:
# строим сводную таблицу с количеством поездок в будний или выходной день
melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count'
)

Weekend,0,1
Regionname,,
Eastern Metropolitan,447,1024
Eastern Victoria,13,40
Northern Metropolitan,1258,2632
Northern Victoria,11,30
South-Eastern Metropolitan,123,327
Southern Metropolitan,1534,3161
Western Metropolitan,960,1988
Western Victoria,8,24


In [10]:
# смотрим на то как выглядит сводная таблица с двумя агрегирующими методами
melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
)

median                          mean              \
Type                        house townhouse   unit        house   townhouse   
Regionname                                                                    
Eastern Metropolitan        674.0     233.5  203.0   717.422847  269.440678   
Eastern Victoria            843.0       0.0  230.0  3108.960000    0.000000   
Northern Metropolitan       459.5     134.0    0.0   619.249092  317.325733   
Northern Victoria           724.0       0.0    0.0  3355.463415    0.000000   
South-Eastern Metropolitan  630.5     240.0  199.0   664.306701  212.160000   
Southern Metropolitan       586.0     246.0    0.0   569.643881  278.858824   
Western Metropolitan        531.0     198.0   62.0   507.883406  244.560669   
Western Victoria            599.5       0.0    0.0   655.500000    0.000000   

                                        
Type                              unit  
Regionname                              
Eastern Metropolitan        330.444444  
Eastern Victoria            295.333333  
Northern Metropolitan       495.026538  
Northern Victoria             0.000000  
South-Eastern Metropolitan  357.864865  
Southern Metropolitan       466.380245  
Western Metropolitan        557.637232  
Western Victoria              0.000000

## Многомерные сводные таблицы

In [11]:
# отобразим сразу два индекса
melb_df.pivot_table(
    values='Price',
    index=['Method', 'Type'],
    columns='Regionname', 
    aggfunc='median',
    fill_value=0
)

Regionname        Eastern Metropolitan  Eastern Victoria  \
Method Type                                                
PI     house                 1244000.0          780000.0   
       townhouse              760000.0               0.0   
       unit                   650000.0               0.0   
S      house                 1127000.0          675000.0   
       townhouse              828000.0               0.0   
       unit                   645750.0          492000.0   
SA     house                  932500.0          950000.0   
       townhouse              807500.0               0.0   
       unit                        0.0               0.0   
SP     house                 1050000.0          672500.0   
       townhouse              910000.0               0.0   
       unit                   515000.0          400000.0   
VB     house                 1100000.0          712500.0   
       townhouse              892500.0               0.0   
       unit                   500000.0               0.0   

Regionname        Northern Metropolitan  Northern Victoria  \
Method Type                                                  
PI     house                   900000.0           500000.0   
       townhouse               632500.0                0.0   
       unit                    410000.0                0.0   
S      house                   920000.0           555000.0   
       townhouse               750000.0                0.0   
       unit                    525500.0                0.0   
SA     house                   817500.0           540000.0   
       townhouse               425000.0                0.0   
       unit                    616000.0                0.0   
SP     house                   900000.0           521000.0   
       townhouse               690000.0                0.0   
       unit                    470000.0                0.0   
VB     house                  1050000.0           690000.0   
       townhouse               640000.0                0.0   
       unit                    450000.0                0.0   

Regionname        South-Eastern Metropolitan  Southern Metropolitan  \
Method Type                                                           
PI     house                        865000.0              1725000.0   
       townhouse                   1190000.0              1055000.0   
       unit                         525000.0               571250.0   
S      house                        883300.0              1611000.0   
       townhouse                    875000.0              1135000.0   
       unit                         606000.0               655000.0   
SA     house                        880000.0              1390000.0   
       townhouse                         0.0              1141000.0   
       unit                              0.0               580000.0   
SP     house                        770000.0              1521750.0   
       townhouse                    800000.0              1162500.0   
       unit                         601000.0               550000.0   
VB     house                        850000.0              1800000.0   
       townhouse                         0.0              1250000.0   
       unit                         700000.0               500000.0   

Regionname        Western Metropolitan  Western Victoria  
Method Type                                               
PI     house                  870000.0          630000.0  
       townhouse              670000.0               0.0  
       unit                   360000.0               0.0  
S      house                  870000.0          397500.0  
       townhouse              729000.0               0.0  
       unit                   489000.0               0.0  
SA     house                  772500.0               0.0  
       townhouse              467500.0               0.0  
       unit                   571000.0               0.0  
SP     house                  865000.0          360000.0  
       t

## Доступ к данным в сводной таблице

In [3]:
# готовим сводную таблицу для того чтобы смотреть нужные данные
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type', 
    aggfunc=['median', 'mean'],
    fill_value=0
)

# получаем обьект Multiindex
pivot.columns



MultiIndex([('median',     'house'),
            ('median', 'townhouse'),
            ('median',      'unit'),
            (  'mean',     'house'),
            (  'mean', 'townhouse'),
            (  'mean',      'unit')],
           names=[None, 'Type'])

In [17]:
# просматриваем признак сводной таблицы
pivot['mean']['unit']

Regionname
Eastern Metropolitan          330.444444
Eastern Victoria              295.333333
Northern Metropolitan         495.026538
Northern Victoria               0.000000
South-Eastern Metropolitan    357.864865
Southern Metropolitan         466.380245
Western Metropolitan          557.637232
Western Victoria                0.000000
Name: unit, dtype: float64

In [20]:
# обрящаемся к столбацам как 'матрёшка' для фильтрации данных
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

# получаем индексы отфильтрованой таблицы
display(list(filtered_pivot.index))

median                        mean              \
Type                   house townhouse  unit       house   townhouse   
Regionname                                                             
Southern Metropolitan  586.0     246.0   0.0  569.643881  278.858824   
Western Metropolitan   531.0     198.0  62.0  507.883406  244.560669   

                                   
Type                         unit  
Regionname                         
Southern Metropolitan  466.380245  
Western Metropolitan   557.637232

['Southern Metropolitan', 'Western Metropolitan']

In [ ]:
# смотрим на ручное создание сводной таблицы
import numpy as np
import pandas as pd

mser = pd.Series(
    np.random.rand(8),
    index=[['white', 'white', 'white', 'blue', 'blue', 'red', 'red', 'red'],
           ['up', 'down', 'right', 'up', 'down', 'up', 'down', 'left']]
)

display(mser)

# смотрим на иерархию индексов
mser.index

white  up       0.816846
       down     0.948356
       right    0.215319
blue   up       0.981022
       down     0.519215
red    up       0.189936
       down     0.340134
       left     0.616020
dtype: float64

MultiIndex([('white',    'up'),
            ('white',  'down'),
            ('white', 'right'),
            ( 'blue',    'up'),
            ( 'blue',  'down'),
            (  'red',    'up'),
            (  'red',  'down'),
            (  'red',  'left')],
           )

In [36]:
import numpy as np
import pandas as pd



# видим вложенные признаки в сводной таблице
mframe = pd.DataFrame(
    np.random.rand(16).reshape(4, 4),
    index=[['white','white', 'red', 'red'],
           ['up', 'down', 'up', 'down']],
    columns=[['pen', 'pen', 'paper', 'paper'], [1, 2, 1, 2]]
)

display(mframe)

pen               paper          
                   1         2         1         2
white up    0.201415  0.654990  0.712577  0.557007
      down  0.817623  0.786275  0.805871  0.815839
red   up    0.479698  0.685352  0.112269  0.740280
      down  0.655438  0.351476  0.991015  0.220156